# 예제 3.29-3.33: 다중 선형 회귀

## 학습목표
1. **다중 선형 회귀** 개념 이해하기 - 여러 입력으로 여러 출력 예측
2. **DataLoader와 TensorDataset** 사용법 익히기
3. **배치(Batch) 학습** 방법 이해하기
4. **미니배치 경사하강법** 구현하기

---

#### 예제 3.29-3.30 데이터 및 라이브러리 준비

**다중 선형 회귀 (Multiple Linear Regression)**
- 여러 개의 입력 변수로 출력을 예측
- 예: 2개 입력 → 2개 출력

In [ ]:
import torch
from torch import nn
from torch import optim
# TensorDataset: 텐서를 데이터셋으로 변환
# DataLoader: 배치 단위로 데이터를 불러오는 유틸리티
from torch.utils.data import TensorDataset, DataLoader 

# 훈련 데이터: 입력 2개, 출력 2개
# train_x shape: (6, 2) - 6개 샘플, 각 샘플당 2개 특성
train_x = torch.FloatTensor([
    [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7]
])
# train_y shape: (6, 2) - 6개 샘플, 각 샘플당 2개 타겟
train_y = torch.FloatTensor([
    [0.1, 1.5], [1, 2.8], [1.9, 4.1], [2.8, 5.4], [3.7, 6.7], [4.6, 8]
])

---

#### 예제 3.31 데이터셋과 데이터로더 정의

**DataLoader 매개변수**
- `batch_size`: 한 번에 처리할 샘플 수
- `shuffle`: 에포크마다 데이터 순서 섞기 (과적합 방지)
- `drop_last`: 마지막 불완전한 배치 버리기

In [ ]:
# TensorDataset: (입력, 타겟) 쌍으로 데이터셋 생성
train_dataset = TensorDataset(train_x, train_y)

# DataLoader: 배치 단위로 데이터 제공
# batch_size=2: 한 번에 2개 샘플씩 처리
# shuffle=True: 매 에포크마다 데이터 순서 무작위화
# drop_last=True: 배치 크기에 맞지 않는 마지막 데이터 버림
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=True)

---

#### 예제 3.32 모델 정의

**nn.Linear(in_features, out_features, bias)**
- in_features=2: 입력 차원 (2개 특성)
- out_features=2: 출력 차원 (2개 타겟)
- bias=True: 편향 사용

In [ ]:
# 모델: 입력 2개 → 출력 2개 (bias=True로 편향 사용)
model = nn.Linear(2, 2, bias=True)

# 손실함수: MSE
criterion = nn.MSELoss()

# 옵티마이저: SGD
optimizer = optim.SGD(model.parameters(), lr=0.001)

---

#### 예제 3.33 미니배치 학습

**미니배치 경사하강법**
- 전체 데이터를 작은 배치로 나누어 학습
- 메모리 효율적, 더 빠른 수렴 가능

In [ ]:
# 미니배치 학습 루프
for epoch in range(20000):
    cost = 0.0  # 에포크별 총 손실
    
    # DataLoader가 배치 단위로 데이터 제공
    for batch in train_dataloader:
        x, y = batch  # 배치에서 입력과 타겟 분리
        output = model(x)  # 순전파
        
        loss = criterion(output, y)  # 손실 계산
        
        optimizer.zero_grad()  # 기울기 초기화
        loss.backward()        # 역전파
        optimizer.step()       # 파라미터 업데이트
        
        cost += loss  # 배치 손실 누적

    # 평균 손실 계산 (총 손실 / 배치 수)
    cost = cost / len(train_dataloader)
    
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")